# Exploring Reliability with APC Data
In this workbook we will explore a dataset of arrival and departure information as well as passenger load information using data shared by Calgary Transit.

**Please Note**: This data has been shared by Calgary Transit for education and research purposes only. The data is not to be used or shared outside of the coursework and exercises we conduct in this class.

To access the dataset, download the `.zip` file on Quercus entitled "Lab 2 - Calgary APC Data", unzip, and place the folder titled `calgary_APC_2019-01-06_2019-01_27` in the data folder of this repository.

This workbook makes some initial use of the Altair module. We will get more in depth with Altair later, but I suggest taking a look at making [simple charts with Altair](https://altair-viz.github.io/altair-tutorial/notebooks/02-Simple-Charts.html) if you are new.

## Data Structure
![Database Overview](img/patterns_db_overview.png)

The data provided is the output of a relational database, with each table exported as a separate file (much like the way GTFS data is typically shared). In this database there are a number of files, of which we will use a subset. In our case we are interested in:
* `DETAIL`, which contains the core events (stopping and starting of buses) that we are looking to analyse to learn about reliability.
* `SAMPLE`, which contains information about samples of bus files. A single sample contains multiple details for a single bus (and therefore route)
* `ROUTE`, which is a named collection of various patterns that service an area. The `ROUTE` table contains various pieces of information about a specfic route.
* `STOP`, which contains information about specific stops that are served.

In this dataset, the suffix `-HND` is used to denote a unique id. For example, `DETAILHND` is a unique identifier for each detail event, while `SAMPLEHND` is a unique identifier for each sample in the database.

For specific details on the column meanings in each of these datasets, please refer to the reference guide posted on Quercus.

In [1]:
import os
import pandas as pd
import altair as alt # For plotting

data_folder = "../data/calgary_APC_2019-01-06_2019-01-27"

Our first task is to load in the relevant tables that we'll need to do some of this analysis, and convert fields that need converting. In our case most of the important data we need is in the `DETAIL` table, however we will also need some relevant information from other tables in order to learn about trips, routes, and schedules.

From the data documentation we also know that the `DETAILTYPE` contains important information about the type of stop, and that `P` represents actual passenger activity and may be the most useful for our purposes.

What we are specifically interested in is **schedule deviation**, and thankfully the data that we are provided contains a specific column with both the start and end times, but also the start and end time variation from the schedule (confirming this using the `SCHEDULE` file is left as an exercise). To follow our convention we'll convert the time variations to minutes, and flip the sign so that early times are negative.

In [2]:
# Start with the big file
detail = pd.read_csv(os.path.join(data_folder, "DETAIL.csv"))

# Convert our actual date times to Python datetimes
detail['DETAILSTART'] = pd.to_datetime(detail['DETAILSTART'])
detail['DETAILEND'] = pd.to_datetime(detail['DETAILEND'])

# Convert to minutes and flip the sign so that early times are negative
detail['startvar_min'] = -1 * detail['DETAILSTARTVAR'] / 60
detail['endvar_min'] = -1 * detail['DETAILENDVAR'] / 60

# Other supporting tables as needed
sample = pd.read_csv(os.path.join(data_folder, "SAMPLE.csv"))
route = pd.read_csv(os.path.join(data_folder, "ROUTE.csv"), dtype={'ROUTEMAJOR': 'Int64'})
stop = pd.read_csv(os.path.join(data_folder, "STOP.csv"))

# Print some basic stats about the size of the data we're working with
print(f"There are {detail.shape[0]} observations")
print(f"There are {detail[detail.DETAILTYPE == 'P'].shape[0]} actual stop observations")

There are 3126819 observations
There are 1647510 actual stop observations


## Initial Exploration
Let's look at our newly created columns for schedule variation on arrival and departure and start to get a bit of an idea of what is going on under the surface. We can use the `.describe()` built-in Pandas function to get some basic stats.

In [13]:
# Filter down to only actual stop observations (if desired). See database documentation as needed.
detail = detail[(detail.DETAILTYPE == 'P') & (detail.DETAILPROC == 0)]
detail[['startvar_min', 'endvar_min']].describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,startvar_min,endvar_min
count,1.64751e+06,1.64751e+06
mean,0.521813,1.2507
std,4.61683,4.39888
min,-1439.75,-74.5
25%,-1.11667,-0.583333
50%,0.183333,0.616667
75%,1.81667,2.38333
max,110.883,111.183


While we can learn some things from the moments, it might be more useful to plot a quick histogram of these values to understand a little bit how they look. Given the size of the dataset, however, it is a good idea in our exploratory case to take a sample of the set with `.sample(n)`

In [3]:
# Distribution of variations using a sample. Change 'maxbins' to get different size bins.
alt.Chart(detail.sample(5000)).mark_bar().encode(
    alt.X("startvar_min:Q", bin=alt.Bin(maxbins=200), title="Schedule Deviation (min)"),
    alt.Y('count()', title="Number of Records"),
)

alt.Chart(...)

Based on this distribution, some of the summary information, and our knowledge about transit operations and bus bunching, we can set a reasonable threshold for schedule deviation values. A range between 10 minutes early and 20 minutes late should capture the vast majority of normal operating buses while still leaving room for potential lateness.

In [5]:
# Apply the filter of outliers here
start_count = detail.shape[0]
clean_detail = detail[(detail.startvar_min > -10) & (detail.startvar_min < 20)]
print(f"Removed {start_count - clean_detail.shape[0]} rows ({100 * (start_count - clean_detail.shape[0])/ start_count}%)")

Removed 44607 rows (1.4265936083924269%


## Route-Level Analysis

To analyse a single route, we need to isolate the representative `ROUTEHND` values for a particular route. Quite often there will be two major routes (for the two directions) with lots of data, and two other routes that represent partial runs of the route. To find out which ones have the most data and are the most useful for our purpose, let's get some information about the data available for all `ROUTEHND` values.

In [4]:
# Start with the actual route number
route_number = 1
# Isolate the route rows that match that route number
route_ids = route[route.ROUTEMAJOR == route_number]
# Bring in stop information that can help us make a decision
route_ids = pd.merge(route_ids, stop[['STOPHND', 'STOPWAY']], left_on='STARTSTOP', right_on='STOPHND')
# Display the information to provide context
display(route_ids[['ROUTEHND', 'ROUTEMAJOR', 'ROUTENAME', 'STARTSTOP', 'STOPWAY', 'ENDSTOP']])
# Bring in the sample data to get a sense of the number of data files available
route_sample = pd.merge(sample[sample.ROUTEHND.isin(route_ids.ROUTEHND)], route, on='ROUTEHND')
# Summarize the counts of various samples
route_sample[['ROUTEHND', 'SAMPLEHND']].groupby('ROUTEHND', as_index=False).count()

,ROUTEHND,ROUTEMAJOR,ROUTENAME,STARTSTOP,STOPWAY,ENDSTOP
0,0,1,1 FOREST LAWN 1-20532-1,1749,Bowness Terminal NW,1779
1,3,1,1 FOREST LAWN 1-20532-4,1749,Bowness Terminal NW,436
2,1,1,1 FOREST LAWN 1-20532-2,1779,NB @ Penbrooke Terminal SE,1749
3,2,1,1 FOREST LAWN 1-20532-3,1779,NB @ Penbrooke Terminal SE,1369
4,4,1,1 FOREST LAWN 1-20532-5,1369,WB 6 AV SW @ 2 ST SW,1749
5,5,1,1 FOREST LAWN 1-20532-6,436,SB 1st ST SW @ 7th AV SW,1779
6,6,1,1 FOREST LAWN 1-20532-7,1763,EB Kensington RD @ Crowchild TR NW,436


,ROUTEHND,SAMPLEHND
0,0,362
1,1,359
2,2,9
3,3,1
4,6,10


Now we want to perform some summaries and calculate some metrics for a given route. Let's start with calculating route-level on-time performance before summarizing that on-time performance by date and stop number. This will let us get an idea of both the data coverage and let us spot any problem stops (both in terms of possible data issues and also data coverage)

In [8]:
# Specify the ROUTEHND we determined above
route_id = 0
# Get all the details for this route
sample_detail = pd.merge(clean_detail[clean_detail.DETAILTYPE.isin(['P']) & (clean_detail.DETAILPROC == 0)], sample[sample.ROUTEHND == route_id], on='SAMPLEHND', how='inner')
# Let's convert the DETAILEND column (departure) to time as our reference:
sample_detail['departure'] = pd.to_datetime(sample_detail.DETAILEND)
# Quick dataset OTP calculation:
on_time = sample_detail[(sample_detail.endvar_min <= 5) & (sample_detail.endvar_min >= -2)].departure.count()
print("On Time:", on_time)
print("  Total:", sample_detail.departure.count())
print("OTP (%):", 100 * on_time/sample_detail.departure.count())

# Let's create a column with the date only to make grouping easier:
sample_detail['date'] = pd.to_datetime(sample_detail['departure'].dt.date)

# Now we find aggregate OTP by date and stop sequence
# First make a grouped dataframe counting only on-time stops made
in_otp = sample_detail[(sample_detail.endvar_min <= 5) & (sample_detail.endvar_min >= -2)][['DETAILHND', 'date', 'ROUTESEQ']].groupby(
    ['date', 'ROUTESEQ'], as_index=False
    ).count()
# Now do the same thing but for all vehicles
all_otp = sample_detail[['DETAILHND', 'date', 'ROUTESEQ']].groupby(['date', 'ROUTESEQ'], as_index=False).count()
# Join them together and rename the columns
otp = pd.merge(in_otp, all_otp, on=['date', 'ROUTESEQ'])
otp.columns = ['date', 'ROUTESEQ', 'in_otp', 'all_counts']
# Final OTP calculation
otp['otp'] = 100 * otp['in_otp'] / otp['all_counts']
otp

On Time: 7689
  Total: 10365
OTP (%): 74.18234442836469


,date,ROUTESEQ,in_otp,all_counts,otp
0,2019-01-07,0,5,10,50.000000
1,2019-01-07,2,3,4,75.000000
2,2019-01-07,3,2,4,50.000000
3,2019-01-07,4,3,4,75.000000
4,2019-01-07,5,1,2,50.000000
...,...,...,...,...,...
1289,2019-01-26,65,5,13,38.461538
1290,2019-01-26,66,6,10,60.000000
1291,2019-01-26,67,4,7,57.142857
1292,2019-01-26,68,3,7,42.857143


Given our method of filtering out only stops made (and ignoring stops passed), we may have some places where data coverage is thin. A representative OTP calcualtion (or other metric) might not be useful with only one or two observations in a day. To visualize the coverage, let's take a look at the dataset but only keep dates and stops where there are 10 or more observations.

In [18]:
to_plot = otp[otp.all_counts >= 10]
print(to_plot.all_counts.max())
alt.Chart(to_plot).mark_circle().encode(
    alt.Y('monthdate(date):T', title='Date of Operation'),
    alt.X('ROUTESEQ:O', title='Stop Sequence'),
    alt.Color('otp:Q', title="OTP (%)"), 
    alt.Size('all_counts:Q', scale=alt.Scale(range=[20, 300]), title="Counts")
)

27


alt.Chart(...)

As a final step to prepare for future visualizations, let's save a copy of the cleaned-up dataset of Route 3 Southbound, keeping stop-level information and variations. We will also include information about passenger loading and demand

In [30]:
# Specify the ROUTEHND we determined above
route_id = 195
# Get all the details for this route
sample_detail = pd.merge(clean_detail[clean_detail.DETAILTYPE.isin(['P']) & (clean_detail.DETAILPROC == 0)], sample[sample.ROUTEHND == route_id], on='SAMPLEHND', how='inner')
# Bring in the stop information
sample_detail = pd.merge(sample_detail, stop, on='STOPHND')
# Let's keep only the columns we need and rename them to be less aggressive with the caps
to_file = sample_detail[['DETAILHND', 'DETAILSTART', 'DETAILEND', 'startvar_min', 'endvar_min', 'IN0', 'OUT0', 'BUSLOAD', 'ROUTESEQ', 'STOPID', 'STOPWAY', 'STOPLONG', 'STOPLAT']].copy()
to_file.columns = ['detail_id', 'start_time', 'end_time', 'start_delta', 'end_delta', 'ons', 'offs', 'theta', 'stop_seq', 'stop_id', 'stopway', 'stop_long', 'stop_lat']
to_file.to_csv('../data/calgary_apc_clean_2019-01-06_2019-01-27.csv', index=False)

## Exercises
* Copy and modify the OTP calculation above to instead measure punctuality (average schedule deviation) and create a similar chart
* Calculate the weighted delay index for route 3. Start by assuming a scheduled headway of 10 minutes. If you'd like, you can download the [Calgary GTFS feed from that time](https://transitfeeds.com/p/calgary-transit/238/20190108) to estimate the scheduled headway for a given time period.
* Using the cleaned-up data in the `to_file` variable we created above, calculate the excess wait observed on the route for all stops for a given day.
* Using the `ons` (boardings) column from the `to_file` dataframe, estiamte the total excess passenger wait observed on the route for a given day.
* Given that not all stops or all buses are captured with 100% certainty, how might you estimate the totall excess boarding passenger wait on the route?